In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['figure.dpi'] = 500
plt.rcParams['text.color'] = 'w'
plt.rcParams['axes.labelcolor'] = 'k'
plt.rcParams['xtick.color'] = 'w'
plt.rcParams['ytick.color'] = 'w'
plt.rcParams['axes.edgecolor'] = 'w'
plt.rcParams['figure.facecolor'] = 'k'
plt.rcParams['axes.facecolor'] = 'k'
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import LogNorm
from matplotlib.patches import FancyArrowPatch
import matplotlib.colors as colors
from astropy.visualization import SqrtStretch, LinearStretch, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import pandas as pd
import re
from astropy.io import fits
import os
from scipy.ndimage import map_coordinates
from scipy.stats import binned_statistic
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from pathlib import Path
from scipy.stats import gaussian_kde
import seaborn as sns
import plotly.graph_objects as go
from scipy.interpolate import griddata

from matplotlib.lines import Line2D
os.chdir("/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling")

In [ ]:
df = pd.read_csv('/Volumes/T7 Shield/Simulation Output/System 1/Sim 1.csv', header=0)

In [ ]:
# Fill missing values in pos_rms column with 1
df['pos_rms'] = df['pos_rms'].fillna(1)
df['mag_rms'] = df['mag_rms'].fillna(6000)

In [ ]:
df

In [ ]:
df['PERT_$\\kappa$'].unique()

In [ ]:
x = df['PERT_$\gamma$'].values
y = df['PERT_$\kappa$'].values
z = df['pos_rms'].values

# Remove NaN values
mask = ~np.isnan(z)
x = x[mask]
y = y[mask]
z = z[mask]

# Create interpolation grid
xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate z values on regular grid
zi = griddata((x, y), z, (xi, yi), method='linear')

In [ ]:
vmin = np.nanmin(zi[zi>0])
vmax = np.nanmax(zi)
norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())

In [ ]:
# Create the contour plot
plt.figure(figsize=(12, 8))

# Create the contour plot using the same data
contour = plt.contourf(xi, yi, zi, levels=100, cmap='inferno', norm=norm)

# Add a colorbar
plt.colorbar(contour, label='Position RMS')

# Set labels and title
plt.xlabel('Shear Strength', color = 'w')
plt.ylabel('Shear Kappa', color = 'w')
plt.title('Shear Strength and Kappa Parameter Space', color = 'w')

# Set the style for dark theme
plt.style.use('dark_background')
plt.grid(True, alpha=0.2)

# Show the plot
plt.show()


In [ ]:
one_sigma = 0.001
sigma_levels = [1, 3, 5]

# Create a figure with subplots
fig, axes = plt.subplots(1, len(sigma_levels), figsize=(15, 5), sharey=True)

for i, ax in enumerate(axes):
    # Filter data based on sigma level
    mask = (df['pos_rms'] <= one_sigma * sigma_levels[i])
    filtered_df = df[mask]
    
    # Create grid for contour plot
    xi = np.linspace(filtered_df['PERT_$\gamma$'].min(), filtered_df['PERT_$\gamma$'].max(), 100)
    yi = np.linspace(filtered_df['PERT_$\kappa$'].min(), filtered_df['PERT_$\kappa$'].max(), 100)
    xi, yi = np.meshgrid(xi, yi)
    
    # Interpolate z values
    zi = griddata((filtered_df['PERT_$\gamma$'], filtered_df['PERT_$\kappa$']), 
                 filtered_df['pos_rms'], (xi, yi), method='cubic')
    
    # Create contour plot using the common norm
    contour = ax.contourf(xi, yi, zi, levels=100, cmap='inferno', norm=norm)
    
    ax.set_title(f'{sigma_levels[i]} $\sigma$', color='w')
    ax.set_xlabel('Shear Strength', color='w')
    if i == 0:
        ax.set_ylabel('Shear Kappa', color='w')
    ax.set_facecolor('white')
    ax.tick_params(colors='w')
    ax.grid(True, alpha=0.2)

# Add common colorbar
cbar = fig.colorbar(contour, ax=axes, label='Position RMS')
cbar.ax.yaxis.label.set_color('w')
cbar.ax.tick_params(colors='w')

plt.show()